In [51]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation, ArtistAnimation, PillowWriter
import pandas as pd
from pandas import HDFStore,DataFrame

In [5]:
f_path = "/mnt/comp_micro/Projects/nucleolus/labelfree_segmentation/object_classification_features.h5"
f = h5py.File(f_path, 'r') #read-only option
print("Keys: %s" % f.keys())

Keys: KeysView(<HDF5 file "object_classification_features.h5" (mode r)>)


In [6]:
pd_data = pd.read_hdf(f_path)

/mnt/data_lg/andrew/miniconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'User Label'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/mnt/data_lg/andrew/miniconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Predicted Class'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/mnt/data_lg/andrew/miniconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Probability of Label 1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this obje

In [7]:
pd_data.head()

,object_id,timestep,labelimage_oid,User Label,Predicted Class,Probability of Label 1,Probability of Label 2,Size in pixels,Center of the object_0,Center of the object_1,Bounding Box Minimum_0,Bounding Box Minimum_1,Bounding Box Maximum_0,Bounding Box Maximum_1
0,0,0,1,0,Label 1,0.72,0.28,273.0,460.772888,4.161172,447.0,0.0,480.0,11.0
1,1,0,2,0,Label 1,0.84,0.16,457.0,411.851196,18.295404,401.0,6.0,424.0,30.0
2,2,0,3,0,Label 2,0.45,0.55,138.0,339.079712,41.978260,334.0,35.0,344.0,49.0
3,3,0,4,0,Label 2,0.41,0.59,15.0,676.333313,43.266666,674.0,42.0,678.0,45.0
4,4,0,5,0,Label 2,0.36,0.64,133.0,369.676697,65.127823,365.0,58.0,375.0,72.0


In [8]:
print(list(f.keys()))
print(list(f.values()))

['images', 'table']
[<HDF5 group "/images" (1631 members)>, <HDF5 dataset "table": shape (1631,), type "|V74">]


In [9]:
images = f["images"]
table = f["table"]

In [10]:
table.dtype


dtype([('object_id', '<i8'), ('timestep', '<i8'), ('labelimage_oid', '<i8'), ('User Label', 'S7'), ('Predicted Class', 'S7'), ('Probability of Label 1', '<f4'), ('Probability of Label 2', '<f4'), ('Size in pixels', '<f4'), ('Center of the object_0', '<f4'), ('Center of the object_1', '<f4'), ('Bounding Box Minimum_0', '<f4'), ('Bounding Box Minimum_1', '<f4'), ('Bounding Box Maximum_0', '<f4'), ('Bounding Box Maximum_1', '<f4')])

## Object Quantification

In [33]:
path722 = "/Volumes/comp_micro/Projects/nucleolus/labelfree_segmentation/722_features-0_table.csv"
nuc_722 = pd.read_csv(path722)

In [34]:
#Only keeping Label 1 objects corresponding to segmented nucleolus
nuc_722_f = nuc_722[nuc_722["Predicted Class"]=="Label 1"]

In [35]:
density = nuc_722_f["Mean Intensity_0"]
density_n = nuc_722_f["Mean Intensity in neighborhood_0"]

In [36]:
nuc_722_f["Mean Intensity_0"] = np.interp(density, (density.min(), density.max()), (-0.5, +2))
nuc_722_f["Mean Intensity in neighborhood_0"] = np.interp(density_n, (density_n.min(), density_n.max()), (-0.5, +2))

/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
nuc_722_f["Aspect Ratio"] = nuc_722_f["Radii of the object_0"]/nuc_722_f["Radii of the object_1"]

/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
#Only keeping columns of interest
nuc_722_f = nuc_722_f[['timestep',  'Size in pixels', 'Aspect Ratio',
                       'Mean Intensity_0', 'Mean Intensity in neighborhood_0']]

In [39]:
#Converting size to microns
nuc_722_f["Size in pixels"] = nuc_722_f["Size in pixels"]*(6.45/63)

In [40]:
nuc_722_f.head()

,timestep,Size in pixels,Aspect Ratio,Mean Intensity_0,Mean Intensity in neighborhood_0
2,0,30.100000,3.688112,0.325997,0.147045
3,0,27.130952,7.698977,0.939571,0.149617
4,0,49.654762,3.152703,0.732901,0.132151
6,0,23.035714,1.046104,0.554148,0.015202
7,0,30.919048,1.710274,-0.234431,-0.045874


In [41]:
nuc_722_f_g = nuc_722_f.groupby("timestep")

In [43]:
datamatrix = {}
for time in np.arange(60):
    data = nuc_722_f_g.get_group(time).to_numpy()[:,1:]
    datamatrix[time] = data

In [44]:
#bins 
_, bins0 = np.histogram(datamatrix[0][:,0])
_, bins1 = np.histogram(datamatrix[0][:,1])
_, bins2 = np.histogram(datamatrix[0][:,2])
_, bins3 = np.histogram(datamatrix[0][:,3])

In [56]:
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'

In [57]:
fig, ax = plt.subplots(nrows=2,ncols=2, figsize = (10, 10))
plt.close(fig)

def updateData(i):
    #if curr <=2: return
    ax[0,0].clear()
    ax[0,1].clear()
    ax[1,0].clear()
    ax[1,1].clear()
    
    ax[0,0].set_title("Size")
    ax[0,1].set_title("Aspect Ratio")
    ax[1,0].set_title("Mean Density")
    ax[1,1].set_title("Mean Density in Neighborhood")
    
    ax[0,0].hist(datamatrix[i][:,0], normed = True, bins = bins0, alpha = 0.5)
    ax[0,1].hist(datamatrix[i][:,1], normed = True, bins = bins1, alpha = 0.5)
    ax[1,0].hist(datamatrix[i][:,2], normed = True, bins = bins2, alpha = 0.5)
    ax[1,1].hist(datamatrix[i][:,3], normed = True, bins = bins3, alpha = 0.5)

simulation = FuncAnimation(fig, updateData, interval=200, repeat=False)

In [58]:
name = "Nucleolus_Quantified.mp4"

In [59]:
simulation.save(name, writer = 'ffmpeg')

/Users/andrew.chen/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_axes.py:6521: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  alternative="'density'", removal="3.1")


KeyError: 60